In [74]:
import numpy as np
import pandas as pd
from sklearn.decomposition import PCA
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.model_selection import train_test_split as tts
from sklearn.preprocessing import minmax_scale # normalize data
import os
np.random.seed(69)

In [75]:
path = "archive"
save_path = "data"

new_lab = "Diabetes_binary" #needs to be exactly the same as a category from the csv file
filt = True
normalize = True

In [76]:
data_pd = pd.read_csv(path + '/diabetes_binary_health_indicators_BRFSS2015.csv')
data_param = data_pd.columns

<h3>Generate data</h3>

In [77]:
# Filter data to be 50/50 diabetes/not diabetes

data_len = len(data_pd)
data_1_pd = data_pd[data_pd['Diabetes_binary'] == 1]
diab_len = len(data_1_pd)
nondiab_len = data_len-diab_len

chosen_non_diab = np.random.choice(nondiab_len, diab_len, replace=False)

data_0_pd = data_pd[data_pd['Diabetes_binary'] == 0].reset_index(drop=True)

data_0_pd_chosen = data_0_pd.iloc[chosen_non_diab]

data_concat = pd.concat([data_1_pd, data_0_pd_chosen]).reset_index(drop=True)

data_filtered = data_concat.iloc[np.random.permutation(len(data_concat))].reset_index(drop=True)

if normalize:
    for key in data_filtered.keys():
        if len(data_filtered[data_filtered[key] > 1]) > 0:
            data_filtered[key] = minmax_scale(data_filtered[key])
data_filtered = data_filtered[:70000]

In [78]:
if filt:
    data_pd = data_filtered

labs_ = data_pd[new_lab].values
dat_ = data_pd[data_pd.columns.drop(new_lab)].values

X_train_, X_test_, y_train_, y_test_ = tts(dat_, labs_, test_size=0.1, shuffle=True)

pca_dim2_ = PCA(n_components=2) 
train_pca2_ = pca_dim2_.fit_transform(X_train_, y_train_)
test_pca2_ = pca_dim2_.transform(X_test_)

pca_dim10_ = PCA(n_components=10) 
train_pca10_ = pca_dim10_.fit_transform(X_train_, y_train_)
test_pca10_ = pca_dim10_.transform(X_test_)



new_file = "data_"+new_lab
if filt:
    new_file += '_5050'
if normalize:
    new_file += '_norm'

isExist = os.path.exists(new_file)
if not isExist:
    os.makedirs(new_file)
    print("The new directory is created!")

pd.DataFrame(train_pca2_).to_csv(new_file + '/trn_pca2.csv', index=False)
pd.DataFrame(test_pca2_).to_csv(new_file + '/tst_pca2.csv', index=False)
pd.DataFrame(train_pca10_).to_csv(new_file + '/trn_pca10.csv', index=False)
pd.DataFrame(test_pca10_).to_csv(new_file + '/tst_pca10.csv', index=False)
pd.DataFrame(y_train_).to_csv(new_file + '/trn_labs.csv', index=False)
pd.DataFrame(y_test_).to_csv(new_file + '/tst_labs.csv', index=False)
pd.DataFrame(X_train_).to_csv(new_file + '/trn_all.csv', index=False)
pd.DataFrame(X_test_).to_csv(new_file + '/tst_all.csv', index=False)
pd.DataFrame(X_test_).to_csv(new_file + '/tst_all.csv', index=False)
pd.DataFrame(data_param).to_csv(new_file + '/param.csv', index=False)